# Python 3.10 - Structural Pattern Matching 🎉

**References**:
* [What’s New In Python 3.10](https://docs.python.org/3/whatsnew/3.10.html)
* [PEP 634 -- Structural Pattern Matching: Specification](https://www.python.org/dev/peps/pep-0634/)
* [PEP 635 -- Structural Pattern Matching: Motivation and Rationale](https://www.python.org/dev/peps/pep-0635)
* [PEP 636 -- Structural Pattern Matching: Tutorial](https://www.python.org/dev/peps/pep-0636/)

## Motiviation

Main reason: more elegant and readable code compared to `if ... elif ... else` chains with a lot of type and structure checking.


## Basic Example

In [1]:
endpoint = "service.com"
endpoint = ("service.com", 8080)
endpoint = ("service.com", 22, "ssh")

In [2]:
if isinstance(endpoint, str):
    host = endpoint
    port = 80
    mode = "http"
elif isinstance(endpoint, tuple) and len(endpoint) == 2:
    host, port = endpoint
    mode = "http"
elif isinstance(endpoint, tuple) and len(endpoint) == 3:
    host, port, mode = endpoint
    
print(host, port, mode)

service.com 22 ssh


In [3]:
match endpoint:
    case str(host):
        port = 80
        mode = "http"
    case (str(host), int(port)):
        mode = "http"
    case str(host), int(port), str(mode):
        pass
        
print(host, port, mode)

service.com 22 ssh


## Literal Matching

In [4]:
text = "hello"

match text:
    case "hello":
        print("Got hello")
    case "world":
        print("Got world")
    case _:
        print("unknown")

Got hello


## Sequence Matching

In [5]:
commands = [
    "go up",
    "fight with dragon",
    "quit",
]

for command in commands:
    match command.split():
        case ["go", direction]:
            print(f"Going {direction}")
        case ["fight", "with", monster]:
            print(f"Fighting a huge {monster}")
        case ["quit"]:
            print("Good bye")

Going up
Fighting a huge dragon
Good bye


## Matching multiple values

In [6]:
command = ["drop", "sword", "shield", "food"]

match command:
    case ["drop", *objects]:
        print(f"Dropping objects: {objects}")

Dropping objects: ['sword', 'shield', 'food']


## OR patterns

In [7]:
commands = [
    "up",
    "go up",
]

for command in commands:
    
    match command.split():
        case ["up"] | ["go", "up"]:
            print("Going up")

Going up
Going up


## Matching sub-patterns

In [8]:
commands = [
    "go up",
    "go down",
    "go left",
    "go right",
]

for command in commands:
    
    match command.split():
        case ["go", ("up" | "down" | "left" | "right")]:
            print("Going somewhere, but where?")

Going somewhere, but where?
Going somewhere, but where?
Going somewhere, but where?
Going somewhere, but where?


## Capturing

In [9]:
commands = [
    "go up",
    "go down",
    "go left",
    "go right",
]

for command in commands:
    
    match command.split():
        case ["go", ("up" | "down" | "left" | "right") as direction]:
            print(f"Going {direction}!")

Going up!
Going down!
Going left!
Going right!


## Conditionals

In [10]:
commands = [
    "go up",
    "go down",
    "go left",
    "go right",
    "go elsewhere",
]

DIRECTIONS = ["up", "down", "left", "right"]

for command in commands:
    
    match command.split():
        case ["go", direction] if direction in DIRECTIONS:
            print(f"Going {direction}!")
        case ["go", the_wrong_way]:
            print(f"'{the_wrong_way}' is no place you want go, trust me!")

Going up!
Going down!
Going left!
Going right!
'elsewhere' is no place you want go, trust me!


## Matching Types

In [11]:
from dataclasses import dataclass
from enum import Enum, auto

class Button(Enum):
    LEFT = auto()
    RIGHT = auto()
    MOUSE_WHEEL = auto()


@dataclass
class Click:
    position: tuple[int, int]
    button: Button
        

@dataclass
class KeyPress:
    key: str


def handle_click_event_at(x, y): print(f"Clicked at {x}|{y}")
def open_context_menu_at(x, y): print(f"Context menu at {x}|{y}")
def handle_keypress(key): print(f"Type {key}")

        
# this would have been triggered by a user
event = Click(position=(5, 10), button=Button.LEFT)
# event = Click(position=(5, 10), button=Button.RIGHT)
# event = KeyPress("a")

match event:
    case Click(position=(x, y), button=Button.LEFT):
        handle_click_event_at(x, y)
    case Click(position=(x, y), button=Button.RIGHT):
        open_context_menu_at(x, y)
    case KeyPress(key=k):
        handle_keypress(k)

Clicked at 5|10


## Matching Positional Attributes

In [12]:
from dataclasses import dataclass
from enum import Enum, auto

class Button(Enum):
    LEFT = auto()
    RIGHT = auto()
    MOUSE_WHEEL = auto()


@dataclass
class Click:
    position: tuple[int, int]
    button: Button
        

event = Click(position=(5, 10), button=Button.RIGHT)

match event:
    case Click((x, y), button):
        print(f"Clicked {button} at {x}|{y}")

Clicked Button.RIGHT at 5|10


In [13]:
from dataclasses import dataclass
from enum import Enum, auto

class Button(Enum):
    LEFT = auto()
    RIGHT = auto()
    MOUSE_WHEEL = auto()


class Click:
    def __init__(self, position: tuple[str, str], button: Button) -> None:
        self.position = position
        self.button = button
        

event = Click(position=(5, 10), button=Button.RIGHT)

match event:
    case Click((x, y), button):
        print(f"Clicked {button} at {x}|{y}")

TypeError: Click() accepts 0 positional sub-patterns (2 given)

In [14]:
from dataclasses import dataclass
from enum import Enum, auto

class Button(Enum):
    LEFT = auto()
    RIGHT = auto()
    MOUSE_WHEEL = auto()

    
class Click:
    __match_args__ = ("position", "button")
    
    def __init__(self, position: tuple[str, str], button: Button) -> None:
        self.position = position
        self.button = button
        

event = Click(position=(5, 10), button=Button.RIGHT)

match event:
    case Click((x, y), button):
        print(f"Clicked {button} at {x}|{y}")

Clicked Button.RIGHT at 5|10


## Matching Dicts

In [15]:
requests = [
    {
        "action": "sign-in",
        "username": "timo",
        "password": "1234",
    },
    {
        "action": "sign-in",
        "username": "timo",
        "password": "wrong",
    },
    {
        "action": "sign-out"
    },
]

def check_user(username, password):
    return username == "timo" and password == "1234"


for request in requests:
    
    match request:
        case {"action": "sign-in", "username": str(u), "password": str(p)} if check_user(u, p):
            print(f"Signing in {u} with password {p}")
        case {"action": "sign-in", **rest}:
            print(f"Please provide valid username and password to sign in")
        case {"action": "sign-out"}:
            print(f"Signing out current user")

Signing in timo with password 1234
Please provide valid username and password to sign in
Signing out current user
